In [ ]:
!git lfs install
!git clone https://huggingface.co/oscarfossey/job_classification
!pip install pickle
!pip install spacy
!spacy download fr_core_news_sm
!pip install joblib

In [32]:
global tokenizer_dl, tokenizer
tokenizer_dl = False

def preprocessing_LSTM(texts_array):
    """preprocessing the strings through the array to predict using the predict_tfidf function
    return an array of token inputs"""

    import numpy as np
    import nltk
    import spacy
    import pickle
    nltk.download('stopwords')

    stopwords = nltk.corpus.stopwords.words('french')
    nlp = spacy.load("fr_core_news_sm")
    
    init_shape  = texts_array.shape
    
    def preprocess(text):
      text = text.lower()
      text = text.replace('(', ' ').replace(')', ' ').replace('.', ' ').replace('  ', ' ')  #drop '(', ')', '.'
      text = nlp(text)
      words = [token.lemma_ for sent in text.sents for token in sent if not token.text in set(stopwords)]
      string = ' '.join(words)
      return string

    def tokenization_LSTM(new_offer):
      global tokenizer, tokenizer_dl
      
      from keras.preprocessing.text import Tokenizer
      from keras.preprocessing.sequence import pad_sequences
      MAX_SEQUENCE_LENGTH=250
      MAX_NB_WORDS = 50000
      if not(tokenizer_dl):
        # If to DL the tokenizer only one time
        tokenizer = pickle.load(open("/content/job_classification/LSTM_tokenizer", 'rb'))
        tokenizer_dl = True

      seq = tokenizer.texts_to_sequences([preprocess(new_offer)])
      padded = pad_sequences(seq, maxlen = MAX_SEQUENCE_LENGTH)
      return (padded)

    token_inputs = np.array([tokenization_LSTM(preprocess(txt)) for txt in list(texts_array.flatten())])

    return token_inputs

In [33]:
global LSTM_model_dl, lstm_model
LSTM_model_dl = False

def predict_LSTM(texts_array):
  
  from joblib import load
  import numpy as np
  global LSTM_model_dl, lstm_model

  init_shape  = texts_array.shape

  if not(LSTM_model_dl):
    lstm_model = load(open("/content/job_classification/model_LSTM.joblib", 'rb')) 
    LSTM_model_dl = True

  predictions = np.array([lstm_model.predict(token_input)  for token_input in list(preprocessing_LSTM(texts_array.flatten()))])
  labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'N', 'M']
  predictions = np.array([labels[np.argmax(pred)] for pred in predictions])
  
  return predictions.reshape(init_shape)